In [1]:
import gc
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import gzip
import json
import matplotlib.pyplot as plt

# Reuse your existing functions for data loading and preprocessing
# load_puzzle_data, one_hot_encode, extract_features, preprocess_data

def build_cnn_feature_extractor(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    outputs = Dense(1)(x)  # Add this line to output a single value
    model = Model(inputs=inputs, outputs=outputs)
    return model

def train_cnn_feature_extractor(X_train, y_train, X_val, y_val):
    model = build_cnn_feature_extractor(X_train.shape[1:])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)
    return model

def extract_cnn_features(model, X):
    return model.predict(X)

def train_xgboost(X_train, y_train, X_val, y_val):
    # Pass early_stopping_rounds during XGBRegressor initialization
    model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, early_stopping_rounds=10) 
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)  # Remove early_stopping_rounds from fit
    return model

def train_CNN():
    # Load and preprocess data (reuse your existing code)
    puzzle_data = load_puzzle_data('buggle-training-data/training_data/training_data-750000.gz')
    print(f"Got {len(puzzle_data)} puzzles.")
    matrix_features, outcomes = extract_features(puzzle_data)

    del puzzle_data
    gc.collect()

    X_train, X_val, X_test, y_train, y_val, y_test = preprocess_data(matrix_features, outcomes)
    print(f"Preprocessed {len(X_train)} training examples, {len(X_val)} validation examples, and {len(X_test)} test examples.")

    del matrix_features, outcomes
    gc.collect()

    # Train CNN feature extractor
    cnn_model = train_cnn_feature_extractor(X_train, y_train, X_val, y_val)

    print("Model built.")

    # Extract features using CNN
    X_train_cnn = extract_cnn_features(cnn_model, X_train)
    X_val_cnn = extract_cnn_features(cnn_model, X_val)
    X_test_cnn = extract_cnn_features(cnn_model, X_test)

    # Train XGBoost model
    xgb_model = train_xgboost(X_train_cnn, y_train, X_val_cnn, y_val)

    del X_train, y_train, X_val, y_val
    gc.collect()

    # Make predictions
    y_pred = xgb_model.predict(X_test_cnn)

    # Evaluate the model
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f"Test MAE: {mae}")
    print(f"Test RMSE: {rmse}")

    # Plot feature importance
    plt.figure(figsize=(10, 6))
    xgb.plot_importance(xgb_model)
    plt.title('Feature Importance in XGBoost Model')
    plt.show()

if __name__ == '__main__':
    train_CNN()

ModuleNotFoundError: No module named 'xgboost'